In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import pandas as pd

In [2]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

In [3]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten())

In [4]:
sum_num = 0
rec = []
for l in net:
    if isinstance(l, nn.Sequential):
        for l2 in l:
            if isinstance(l2, nn.Conv2d):
                num = l2.in_channels * l2.out_channels * l2.kernel_size[0] * l2.kernel_size[1]
                sum_num += num
                rec.append([l2.__class__.__name__, num])
print(f'{sum_num} = {round(sum_num / 1000 / 1000, 1)}M')
pd.DataFrame(rec)

1989928 = 2.0M


,0,1
0,Conv2d,11616
1,Conv2d,9216
2,Conv2d,9216
3,Conv2d,614400
4,Conv2d,65536
5,Conv2d,65536
6,Conv2d,884736
7,Conv2d,147456
8,Conv2d,147456
9,Conv2d,34560


In [5]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])
